In [4]:
import numpy as np
import pandas as pd
import gc
import os, sys, random, datetime

In [2]:
def reduce_memory_usage(df, deep=True, verbose=True, categories=True):
    # All types that we want to change for "lighter" ones.
    # int8 and float16 are not include because we cannot reduce
    # those data types.
    # float32 is not include because float16 has too low precision.
    numeric2reduce = ["int16", "int32", "int64", "float64"]
    start_mem = 0
    if verbose:
        start_mem = memory_usage_mb(df, deep=deep)

    for col, col_type in df.dtypes.iteritems():
        best_type = None
        if col_type == "object":
            df[col] = df[col].astype("category")
            best_type = "category"
        elif col_type in numeric2reduce:
            downcast = "integer" if "int" in str(col_type) else "float"
            df[col] = pd.to_numeric(df[col], downcast=downcast)
            best_type = df[col].dtype.name
        # Log the conversion performed.
        if verbose and best_type is not None and best_type != str(col_type):
            print(f"Column '{col}' converted from {col_type} to {best_type}")

    if verbose:
        end_mem = memory_usage_mb(df, deep=deep)
        diff_mem = start_mem - end_mem
        percent_mem = 100 * diff_mem / start_mem
        print(f"Memory usage decreased from"
              f" {start_mem:.2f}MB to {end_mem:.2f}MB"
              f" ({diff_mem:.2f}MB, {percent_mem:.2f}% reduction)")

In [5]:
def seed_everything(seed=0):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)

## Memory Reducer
# :df pandas dataframe to reduce size             # type: pd.DataFrame()
# :verbose                                        # type: bool
def reduce_mem_usage(df, verbose=True):
    numerics = ['int16', 'int32', 'int64', 'float16', 'float32', 'float64']
    start_mem = df.memory_usage().sum() / 1024**2    
    for col in df.columns:
        col_type = df[col].dtypes
        if col_type in numerics:
            c_min = df[col].min()
            c_max = df[col].max()
            if str(col_type)[:3] == 'int':
                if c_min > np.iinfo(np.int8).min and c_max < np.iinfo(np.int8).max:
                    df[col] = df[col].astype(np.int8)
                elif c_min > np.iinfo(np.int16).min and c_max < np.iinfo(np.int16).max:
                    df[col] = df[col].astype(np.int16)
                elif c_min > np.iinfo(np.int32).min and c_max < np.iinfo(np.int32).max:
                    df[col] = df[col].astype(np.int32)
                elif c_min > np.iinfo(np.int64).min and c_max < np.iinfo(np.int64).max:
                    df[col] = df[col].astype(np.int64)  
            else:
                if c_min > np.finfo(np.float16).min and c_max < np.finfo(np.float16).max:
                    df[col] = df[col].astype(np.float16)
                elif c_min > np.finfo(np.float32).min and c_max < np.finfo(np.float32).max:
                    df[col] = df[col].astype(np.float32)
                else:
                    df[col] = df[col].astype(np.float64)    
    end_mem = df.memory_usage().sum() / 1024**2
    if verbose: print('Mem. usage decreased to {:5.2f} Mb ({:.1f}% reduction)'.format(end_mem, 100 * (start_mem - end_mem) / start_mem))
    return df


In [6]:
print('Load Data')
train_df = pd.read_csv('../input/train_transaction.csv')
test_df = pd.read_csv('../input/test_transaction.csv')
test_df['isFraud'] = 0
train_identity = pd.read_csv('../input/train_identity.csv')
test_identity = pd.read_csv('../input/test_identity.csv')
print('Reduce Memory')
train_df = reduce_mem_usage(train_df)
test_df  = reduce_mem_usage(test_df)
train_identity = reduce_mem_usage(train_identity)
test_identity  = reduce_mem_usage(test_identity)

Load Data
Reduce Memory
Mem. usage decreased to 542.35 Mb (69.4% reduction)
Mem. usage decreased to 473.07 Mb (68.9% reduction)
Mem. usage decreased to 25.86 Mb (42.7% reduction)
Mem. usage decreased to 25.44 Mb (42.7% reduction)


In [7]:
def id_split(dataframe):
    
    dataframe['device_name'] = dataframe['DeviceInfo'].str.split('/', expand=True)[0]
    dataframe['device_version'] = dataframe['DeviceInfo'].str.split('/', expand=True)[1]

    dataframe['OS_id_30'] = dataframe['id_30'].str.split(' ', expand=True)[0]
    dataframe['version_id_30'] = dataframe['id_30'].str.split(' ', expand=True)[1]
 
    dataframe['browser_id_31'] = dataframe['id_31'].str.split(' ', expand=True)[0]
    dataframe['version_id_31'] = dataframe['id_31'].str.split(' ', expand=True)[1]

    dataframe['screen_width'] = dataframe['id_33'].str.split('x', expand=True)[0]
    dataframe['screen_height'] = dataframe['id_33'].str.split('x', expand=True)[1]
    dataframe['id_12'] = dataframe['id_12'].map({'Found':1, 'NotFound':0})
    dataframe['id_15'] = dataframe['id_15'].map({'New':2, 'Found':1, 'Unknown':0})
    dataframe['id_16'] = dataframe['id_16'].map({'Found':1, 'NotFound':0})

    dataframe['id_23'] = dataframe['id_23'].map({'TRANSPARENT':4, 'IP_PROXY':3, 'IP_PROXY:ANONYMOUS':2, 'IP_PROXY:HIDDEN':1})

    dataframe['id_27'] = dataframe['id_27'].map({'Found':1, 'NotFound':0})
    dataframe['id_28'] = dataframe['id_28'].map({'New':2, 'Found':1})

    dataframe['id_29'] = dataframe['id_29'].map({'Found':1, 'NotFound':0})

    dataframe['id_35'] = dataframe['id_35'].map({'T':1, 'F':0})
    dataframe['id_36'] = dataframe['id_36'].map({'T':1, 'F':0})
    dataframe['id_37'] = dataframe['id_37'].map({'T':1, 'F':0})
    dataframe['id_38'] = dataframe['id_38'].map({'T':1, 'F':0})

    dataframe['id_34'] = dataframe['id_34'].fillna(':0')
    dataframe['id_34'] = dataframe['id_34'].apply(lambda x: x.split(':')[1]).astype(np.int8)
    dataframe['id_34'] = np.where(dataframe['id_34']==0, np.nan, dataframe['id_34'])
    
    dataframe['id_33'] = dataframe['id_33'].fillna('0x0')
    dataframe['id_33_0'] = dataframe['id_33'].apply(lambda x: x.split('x')[0]).astype(int)
    dataframe['id_33_1'] = dataframe['id_33'].apply(lambda x: x.split('x')[1]).astype(int)
    dataframe['id_33'] = np.where(dataframe['id_33']=='0x0', np.nan, dataframe['id_33'])
    
    for feature in ['id_01', 'id_31', 'id_33', 'id_36']:
        dataframe[feature + '_count_dist'] = dataframe[feature].map(dataframe[feature].value_counts(dropna=False))
    
    dataframe['DeviceType'].map({'desktop':1, 'mobile':0})
    
    dataframe.loc[dataframe['device_name'].str.contains('SM', na=False), 'device_name'] = 'Samsung'
    dataframe.loc[dataframe['device_name'].str.contains('SAMSUNG', na=False), 'device_name'] = 'Samsung'
    dataframe.loc[dataframe['device_name'].str.contains('GT-', na=False), 'device_name'] = 'Samsung'
    dataframe.loc[dataframe['device_name'].str.contains('Moto G', na=False), 'device_name'] = 'Motorola'
    dataframe.loc[dataframe['device_name'].str.contains('Moto', na=False), 'device_name'] = 'Motorola'
    dataframe.loc[dataframe['device_name'].str.contains('moto', na=False), 'device_name'] = 'Motorola'
    dataframe.loc[dataframe['device_name'].str.contains('LG-', na=False), 'device_name'] = 'LG'
    dataframe.loc[dataframe['device_name'].str.contains('rv:', na=False), 'device_name'] = 'RV'
    dataframe.loc[dataframe['device_name'].str.contains('HUAWEI', na=False), 'device_name'] = 'Huawei'
    dataframe.loc[dataframe['device_name'].str.contains('ALE-', na=False), 'device_name'] = 'Huawei'
    dataframe.loc[dataframe['device_name'].str.contains('-L', na=False), 'device_name'] = 'Huawei'
    dataframe.loc[dataframe['device_name'].str.contains('Blade', na=False), 'device_name'] = 'ZTE'
    dataframe.loc[dataframe['device_name'].str.contains('BLADE', na=False), 'device_name'] = 'ZTE'
    dataframe.loc[dataframe['device_name'].str.contains('Linux', na=False), 'device_name'] = 'Linux'
    dataframe.loc[dataframe['device_name'].str.contains('XT', na=False), 'device_name'] = 'Sony'
    dataframe.loc[dataframe['device_name'].str.contains('HTC', na=False), 'device_name'] = 'HTC'
    dataframe.loc[dataframe['device_name'].str.contains('ASUS', na=False), 'device_name'] = 'Asus'

    dataframe.loc[dataframe.device_name.isin(dataframe.device_name.value_counts()[dataframe.device_name.value_counts() < 200].index), 'device_name'] = "Others"
    dataframe['had_id'] = 1
    gc.collect()
    
    return dataframe

In [8]:
train_identity = id_split(train_identity)
test_identity = id_split(test_identity)

In [9]:
test_df.head()

,TransactionID,TransactionDT,TransactionAmt,ProductCD,card1,card2,card3,card4,card5,card6,...,V331,V332,V333,V334,V335,V336,V337,V338,V339,isFraud
0,3663549,18403224,31.953125,W,10409,111.0,150.0,visa,226.0,debit,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0
1,3663550,18403263,49.000000,W,4272,111.0,150.0,visa,226.0,debit,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0
2,3663551,18403310,171.000000,W,4476,574.0,150.0,visa,226.0,debit,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0
3,3663552,18403310,285.000000,W,10989,360.0,150.0,visa,166.0,debit,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0
4,3663553,18403317,67.937500,W,18018,452.0,150.0,mastercard,117.0,debit,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0


In [10]:
#new features trans
def gen_new(train_trans,test_trans):

    # New feature - log of transaction amount.
    train_trans['TransactionAmt_Log'] = np.log1p(train_trans['TransactionAmt'])
    test_trans['TransactionAmt_Log'] = np.log1p(test_trans['TransactionAmt'])

    # New feature - decimal part of the transaction amount.
    train_trans['TransactionAmt_decimal'] = ((train_trans['TransactionAmt'] - train_trans['TransactionAmt'].astype(int)) * 1000).astype(int)
    test_trans['TransactionAmt_decimal'] = ((test_trans['TransactionAmt'] - test_trans['TransactionAmt'].astype(int)) * 1000).astype(int)

    
    # New feature - day of week in which a transaction happened.
    train_trans['Transaction_day_of_week'] = np.floor((train_trans['TransactionDT'] / (3600 * 24) - 1) % 7)
    test_trans['Transaction_day_of_week'] = np.floor((test_trans['TransactionDT'] / (3600 * 24) - 1) % 7)

    # New feature - hour of the day in which a transaction happened.
    train_trans['Transaction_hour'] = np.floor(train_trans['TransactionDT'] / 3600) % 24
    test_trans['Transaction_hour'] = np.floor(test_trans['TransactionDT'] / 3600) % 24
    
    #train_trans['cents'] = np.round(train_trans['TransactionAmt'] - np.floor(train_trans['TransactionAmt']),2 )
    #test_trans['cents'] = np.round(test_trans['TransactionAmt'] - np.floor(test_trans['TransactionAmt']),2 )
    
    
    #New feature - emaildomain with suffix
    emails = {'gmail': 'google', 'att.net': 'att', 'twc.com': 'spectrum', 'scranton.edu': 'other', 'optonline.net': 'other', 'hotmail.co.uk': 'microsoft', 'comcast.net': 'other', 'yahoo.com.mx': 'yahoo', 'yahoo.fr': 'yahoo', 'yahoo.es': 'yahoo', 'charter.net': 'spectrum', 'live.com': 'microsoft', 'aim.com': 'aol', 'hotmail.de': 'microsoft', 'centurylink.net': 'centurylink', 'gmail.com': 'google', 'me.com': 'apple', 'earthlink.net': 'other', 'gmx.de': 'other', 'web.de': 'other', 'cfl.rr.com': 'other', 'hotmail.com': 'microsoft', 'protonmail.com': 'other', 'hotmail.fr': 'microsoft', 'windstream.net': 'other', 'outlook.es': 'microsoft', 'yahoo.co.jp': 'yahoo', 'yahoo.de': 'yahoo', 'servicios-ta.com': 'other', 'netzero.net': 'other', 'suddenlink.net': 'other', 'roadrunner.com': 'other', 'sc.rr.com': 'other', 'live.fr': 'microsoft', 'verizon.net': 'yahoo', 'msn.com': 'microsoft', 'q.com': 'centurylink', 'prodigy.net.mx': 'att', 'frontier.com': 'yahoo', 'anonymous.com': 'other', 'rocketmail.com': 'yahoo', 'sbcglobal.net': 'att', 'frontiernet.net': 'yahoo', 'ymail.com': 'yahoo', 'outlook.com': 'microsoft', 'mail.com': 'other', 'bellsouth.net': 'other', 'embarqmail.com': 'centurylink', 'cableone.net': 'other', 'hotmail.es': 'microsoft', 'mac.com': 'apple', 'yahoo.co.uk': 'yahoo', 'netzero.com': 'other', 'yahoo.com': 'yahoo', 'live.com.mx': 'microsoft', 'ptd.net': 'other', 'cox.net': 'other', 'aol.com': 'aol', 'juno.com': 'other', 'icloud.com': 'apple','uknown':'uknown'}
    us_emails = ['gmail', 'net', 'edu']

    for c in ['P_emaildomain', 'R_emaildomain']:
        train_trans[c] = train_trans[c].fillna('uknown')
        test_trans[c] = test_trans[c].fillna('uknown')
        
        train_trans[c + '_bin'] = train_trans[c].map(emails)
        test_trans[c + '_bin'] = test_trans[c].map(emails)
    
        train_trans[c + '_suffix'] = train_trans[c].apply(lambda x: str(x).split('.')[-1])
        test_trans[c + '_suffix'] = test_trans[c].apply(lambda x: str(x).split('.')[-1])
        
        train_trans[c + '_prefix'] = train_trans[c].apply(lambda x: str(x).split('.')[0])
        test_trans[c + '_prefix'] = test_trans[c].apply(lambda x: str(x).split('.')[0])

        train_trans[c + '_suffix_us'] = train_trans[c + '_suffix'].map(lambda x: x if str(x) not in us_emails else 'us')
        test_trans[c + '_suffix_us'] = test_trans[c + '_suffix'].map(lambda x: x if str(x) not in us_emails else 'us')
    train_trans['email_check'] = np.where((train_trans['P_emaildomain']==train_trans['R_emaildomain'])&(train_trans['P_emaildomain']!='uknown'),1,0)
    test_trans['email_check'] = np.where((test_trans['P_emaildomain']==test_trans['R_emaildomain'])&(test_trans['P_emaildomain']!='uknown'),1,0)
    
    #New feature - dates
    START_DATE = datetime.datetime.strptime('2017-11-30', '%Y-%m-%d')
    from pandas.tseries.holiday import USFederalHolidayCalendar as calendar
    dates_range = pd.date_range(start='2017-10-01', end='2019-01-01')
    us_holidays = calendar().holidays(start=dates_range.min(), end=dates_range.max())

    for df in [train_trans, test_trans]:
        # Temporary
        df['DT'] = df['TransactionDT'].apply(lambda x: (START_DATE + datetime.timedelta(seconds = x)))
        df['DT_M'] = (df['DT'].dt.year-2017)*12 + df['DT'].dt.month
        df['DT_W'] = (df['DT'].dt.year-2017)*52 + df['DT'].dt.weekofyear
        df['DT_D'] = (df['DT'].dt.year-2017)*365 + df['DT'].dt.dayofyear

        df['DT_hour'] = df['DT'].dt.hour
        df['DT_day_week'] = df['DT'].dt.dayofweek
        df['DT_day'] = df['DT'].dt.day
        df['DT_day_month'] = (df['DT'].dt.day).astype(np.int8)
        # Possible solo feature
        df['is_december'] = df['DT'].dt.month
        df['is_december'] = (df['is_december']==12).astype(np.int8)

        # Holidays
        df['is_holiday'] = (df['DT'].dt.date.astype('datetime64').isin(us_holidays)).astype(np.int8)
   
    #New feature - binary encoded 1/0 gen new
    i_cols = ['M1','M2','M3','M5','M6','M7','M8','M9']
    for df in [train_trans, test_trans]:
        df['M_sum'] = df[i_cols].sum(axis=1).astype(np.int8)
        df['M_na'] = df[i_cols].isna().sum(axis=1).astype(np.int8)

    #New feature - ProductCD and M4 Target mean
    for col in ['ProductCD','M4']:
        temp_dict = train_trans.groupby([col])['isFraud'].agg(['mean']).reset_index().rename(columns={'mean': col+'_target_mean'})
        temp_dict.index = temp_dict[col].values
        temp_dict = temp_dict[col+'_target_mean'].to_dict()

        train_trans[col+'_target_mean'] = train_trans[col].map(temp_dict)
        test_trans[col+'_target_mean']  = test_trans[col].map(temp_dict)
    
    #New feature - use it for aggregations
    train_trans['uid1'] = train_trans['card1'].astype(str)+'_'+train_trans['card2'].astype(str) 
    test_trans['uid1'] = test_trans['card1'].astype(str)+'_'+test_trans['card2'].astype(str)

    train_trans['uid2'] = train_trans['uid1'].astype(str)+'_'+train_trans['card3'].astype(str)+'_'+train_trans['card5'].astype(str)
    test_trans['uid2'] = test_trans['uid1'].astype(str)+'_'+test_trans['card3'].astype(str)+'_'+test_trans['card5'].astype(str)

    train_trans['uid3'] = train_trans['uid2'].astype(str)+'_'+train_trans['addr1'].astype(str)+'_'+train_trans['addr2'].astype(str)
    test_trans['uid3'] = test_trans['uid2'].astype(str)+'_'+test_trans['addr1'].astype(str)+'_'+test_trans['addr2'].astype(str)

    # Check if the Transaction Amount is common or not (we can use freq encoding here)
    # In our dialog with a model we are telling to trust or not to these values   
    # Clip Values
    train_trans['TransactionAmt'] = train_trans['TransactionAmt'].clip(0,5000)
    test_trans['TransactionAmt']  = test_trans['TransactionAmt'].clip(0,5000)

    train_trans['TransactionAmt_check'] = np.where(train_trans['TransactionAmt'].isin(test_trans['TransactionAmt']), 1, 0)
    test_trans['TransactionAmt_check']  = np.where(test_trans['TransactionAmt'].isin(train_trans['TransactionAmt']), 1, 0)


    return train_trans,test_trans


In [11]:
train_df,test_df = gen_new(train_df,test_df)

In [12]:
def timeblock_frequency_encoding(train_df, test_df, periods, columns, 
                                 with_proportions=True, only_proportions=False):
    for period in periods:
        for col in columns:
            new_col = col +'_'+ period
            train_df[new_col] = train_df[col].astype(str)+'_'+train_df[period].astype(str)
            test_df[new_col]  = test_df[col].astype(str)+'_'+test_df[period].astype(str)

            temp_df = pd.concat([train_df[[new_col]], test_df[[new_col]]])
            fq_encode = temp_df[new_col].value_counts().to_dict()

            train_df[new_col] = train_df[new_col].map(fq_encode)
            test_df[new_col]  = test_df[new_col].map(fq_encode)
            
            if only_proportions:
                train_df[new_col] = train_df[new_col]/train_df[period+'_total']
                test_df[new_col]  = test_df[new_col]/test_df[period+'_total']

            if with_proportions:
                train_df[new_col+'_proportions'] = train_df[new_col]/train_df[period+'_total']
                test_df[new_col+'_proportions']  = test_df[new_col]/test_df[period+'_total']

    return train_df, test_df

In [13]:
def uid_aggregation(train_df, test_df, main_columns, uids, aggregations):
    for main_column in main_columns:  
        for col in uids:
            for agg_type in aggregations:
                new_col_name = col+'_'+main_column+'_'+agg_type
                temp_df = pd.concat([train_df[[col, main_column]], test_df[[col,main_column]]])
                temp_df = temp_df.groupby([col])[main_column].agg([agg_type]).reset_index().rename(
                                                        columns={agg_type: new_col_name})

                temp_df.index = list(temp_df[col])
                temp_df = temp_df[new_col_name].to_dict()   

                train_df[new_col_name] = train_df[col].map(temp_df)
                test_df[new_col_name]  = test_df[col].map(temp_df)
    return train_df, test_df

def uid_aggregation_and_normalization(train_df, test_df, main_columns, uids, aggregations):
    for main_column in main_columns:  
        for col in uids:
            
            new_norm_col_name = col+'_'+main_column+'_std_norm'
            norm_cols = []
            
            for agg_type in aggregations:
                new_col_name = col+'_'+main_column+'_'+agg_type
                temp_df = pd.concat([train_df[[col, main_column]], test_df[[col,main_column]]])
                temp_df = temp_df.groupby([col])[main_column].agg([agg_type]).reset_index().rename(
                                                        columns={agg_type: new_col_name})

                temp_df.index = list(temp_df[col])
                temp_df = temp_df[new_col_name].to_dict()   

                train_df[new_col_name] = train_df[col].map(temp_df)
                test_df[new_col_name]  = test_df[col].map(temp_df)
                norm_cols.append(new_col_name)
            
            train_df[new_norm_col_name] = (train_df[main_column]-train_df[norm_cols[0]])/train_df[norm_cols[1]]
            test_df[new_norm_col_name]  = (test_df[main_column]-test_df[norm_cols[0]])/test_df[norm_cols[1]]          
            
            del train_df[norm_cols[0]], train_df[norm_cols[1]]
            del test_df[norm_cols[0]], test_df[norm_cols[1]]
                                              
    return train_df, test_df

In [14]:
def frequency_encoding(train_df, test_df, columns, self_encoding=False):
    for col in columns:
        temp_df = pd.concat([train_df[[col]], test_df[[col]]])
        fq_encode = temp_df[col].value_counts(dropna=False).to_dict()
        if self_encoding:
            train_df[col] = train_df[col].map(fq_encode)
            test_df[col]  = test_df[col].map(fq_encode)            
        else:
            train_df[col+'_fq_enc'] = train_df[col].map(fq_encode)
            test_df[col+'_fq_enc']  = test_df[col].map(fq_encode)
    return train_df, test_df

In [15]:
#1. More interaction between card features + fill nans
i_cols = ['TransactionID','card1','card2','card3','card4','card5','card6']

full_df = pd.concat([train_df[i_cols], test_df[i_cols]])

## I've used frequency encoding before so we have ints here
## we will drop very rare cards
full_df['card6'] = np.where(full_df['card6']==30, np.nan, full_df['card6'])
full_df['card6'] = np.where(full_df['card6']==16, np.nan, full_df['card6'])

i_cols = ['card2','card3','card4','card5','card6']

## We will find best match for nan values and fill with it 把23456都补上好多了
for col in i_cols:
    temp_df = full_df.groupby(['card1',col])[col].agg(['count']).reset_index()
    temp_df = temp_df.sort_values(by=['card1','count'], ascending=False).reset_index(drop=True)
    del temp_df['count']
    temp_df = temp_df.drop_duplicates(keep='first').reset_index(drop=True)
    temp_df.index = temp_df['card1'].values
    temp_df = temp_df[col].to_dict()
    full_df[col] = np.where(full_df[col].isna(), full_df['card1'].map(temp_df), full_df[col])
    
    
i_cols = ['card1','card2','card3','card4','card5','card6']
for col in i_cols:
    train_df[col] = full_df[full_df['TransactionID'].isin(train_df['TransactionID'])][col].values
    test_df[col] = full_df[full_df['TransactionID'].isin(test_df['TransactionID'])][col].values


In [14]:
def check_dependency(independent_var, dependent_var):
    
    independent_uniques = []
    temp_df = pd.concat([train_df[[independent_var, dependent_var]], test_df[[independent_var, dependent_var]]])
    
    for value in temp_df[independent_var].unique():
        independent_uniques.append(temp_df[temp_df[independent_var] == value][dependent_var].value_counts().shape[0])

    values = pd.Series(data=independent_uniques, index=temp_df[independent_var].unique())
    
    N = len(values)
    N_dependent = len(values[values == 1])
    N_notdependent = len(values[values > 1])
    N_null = len(values[values == 0])
        
    print(f'In {independent_var}, there are {N} unique values')
    print(f'{N_dependent}/{N} have one unique {dependent_var} value')
    print(f'{N_notdependent}/{N} have more than one unique {dependent_var} values')
    print(f'{N_null}/{N} have only missing {dependent_var} values\n')

In [ ]:
check_dependency('ProductCD','had_id')

In [ ]:
train_df[['ProductCD','had_id']].head(20)

In [15]:
print(train_df['C5'].isnull().sum()/train_df.shape[0])
print(test_df['C5'].isnull().sum()/test_df.shape[0])

0.0
5.920768278891869e-06


In [ ]:
test_df[~test_df['id_31'].isnull()]['C9'].value_counts()

In [ ]:
#1.1 find dependency and fillna
#'dist1', 'C3',只有test有C3的缺失,且只在dist1不缺失的时候缺失，dist1不缺失的时候C3全都是0
test_df['C3'] = test_df['C3'].fillna(0)
#'R_emaildomain', 'C5',只有test有C5的缺失，基本上都是在R_emaildomain不缺失的时候缺失，R_emaildomain缺失的C5缺失只有3个
test_df['C5'] = test_df['C5'].fillna(0)
#'id_30','C7',只有test有C7的缺失，只在id_30不缺失的时候缺失，id_30不缺失的C7缺失只有3个，其他都是0（Device）
test_df['C7'] = test_df['C7'].fillna(0)
#'id_31','C9',只有test有C9的缺失，只在id_31不缺失的时候缺失，id_31不缺失的C9缺失只有3个，其他都是0（Browser）
test_df['C9'] = test_df['C9'].fillna(0)
#

In [16]:
#2. Keep intersactions
for col in ['card1']: 
    valid_card = pd.concat([train_df[[col]], test_df[[col]]])
    valid_card = valid_card[col].value_counts()
    valid_card_std = valid_card.values.std()

    invalid_cards = valid_card[valid_card<=2]
    print('Rare cards',len(invalid_cards))

    valid_card = valid_card[valid_card>2]
    valid_card = list(valid_card.index)

    print('No intersection in Train', len(train_df[~train_df[col].isin(test_df[col])]))
    print('Intersection in Train', len(train_df[train_df[col].isin(test_df[col])]))
    
    train_df[col] = np.where(train_df[col].isin(test_df[col]), train_df[col], np.nan)
    test_df[col]  = np.where(test_df[col].isin(train_df[col]), test_df[col], np.nan)

    train_df[col] = np.where(train_df[col].isin(valid_card), train_df[col], np.nan)
    test_df[col]  = np.where(test_df[col].isin(valid_card), test_df[col], np.nan)
    print('#'*20)

for col in ['card2','card3','card4','card5','card6']: 
    print('No intersection in Train', col, len(train_df[~train_df[col].isin(test_df[col])]))
    print('Intersection in Train', col, len(train_df[train_df[col].isin(test_df[col])]))
    
    train_df[col] = np.where(train_df[col].isin(test_df[col]), train_df[col], np.nan)
    test_df[col]  = np.where(test_df[col].isin(train_df[col]), test_df[col], np.nan)
    print('#'*20)

Rare cards 5993
No intersection in Train 10396
Intersection in Train 580144
####################
No intersection in Train card2 0
Intersection in Train card2 590540
####################
No intersection in Train card3 47
Intersection in Train card3 590493
####################
No intersection in Train card4 0
Intersection in Train card4 590540
####################
No intersection in Train card5 176
Intersection in Train card5 590364
####################
No intersection in Train card6 30
Intersection in Train card6 590510
####################


In [17]:
#3.generate accurate userids and cardids
#uid1=card1_card2
#uid2=card_1235
#uid3=card_1235_addr12
#uid4=card_1235_Pemail
#uid5=card_1235_Remail

train_df['uid4'] = train_df['uid3'].astype(str)+'_'+train_df['P_emaildomain'].astype(str)
test_df['uid4'] = test_df['uid3'].astype(str)+'_'+test_df['P_emaildomain'].astype(str)

train_df['uid5'] = train_df['uid3'].astype(str)+'_'+train_df['R_emaildomain'].astype(str)
test_df['uid5'] = test_df['uid3'].astype(str)+'_'+test_df['R_emaildomain'].astype(str)

train_df['uid6'] = train_df['card1'].astype(str)+'_'+train_df['D15'].astype(str)
test_df['uid6'] = test_df['card1'].astype(str)+'_'+test_df['D15'].astype(str)

#try to generate more accuracy card_id and user_id
#uid1\2 不太有使用的价值了

#guess_card_id
train_df['TransactionDTday'] = (train_df['TransactionDT']/(60*60*24)).map(int)
test_df['TransactionDTday'] = (test_df['TransactionDT']/(60*60*24)).map(int)
train_df['D1minusday'] = train_df['D1'] - train_df['TransactionDTday'] #发卡日
test_df['D1minusday'] = test_df['D1'] - test_df['TransactionDTday']
train_df['D4minusday'] = train_df['D4'] - train_df['TransactionDTday'] #发卡日
test_df['D4minusday'] = test_df['D4'] - test_df['TransactionDTday']

#这个应该对D1\D2\D3\D8有效果,D2没必要动，D3/D8应该有别的用法
train_df['cid_1'] = train_df['uid4'].astype(str)+'_'+train_df['D1minusday'].astype(str)
test_df['cid_1'] = test_df['uid4'].astype(str)+'_'+test_df['D1minusday'].astype(str)

#guess_user_id 用D4
train_df['uid7'] = train_df['uid4'].astype(str)+'_'+train_df['D4minusday'].astype(str)
test_df['uid7'] = test_df['uid4'].astype(str)+'_'+test_df['D4minusday'].astype(str)

print('#'*10)
print('Most common uIds:')
new_columns = ['uid1','uid2','uid3','uid4','uid5','uid6','uid7','cid_1']
for col in new_columns:
    print('#'*10, col)
    print(train_df[col].value_counts()[:10])

# Do Global frequency encoding 

i_cols = ['card1','card2','card3','card5'] + new_columns
train_df, test_df = frequency_encoding(train_df, test_df, i_cols, self_encoding=False)

##########
Most common uIds:
########## uid1
7919_194.0     14891
9500_321.0     14112
15885_545.0    10332
17188_321.0    10312
15066_170.0     7918
12695_490.0     7079
6019_583.0      6766
12544_321.0     6760
2803_100.0      6126
7585_553.0      5325
Name: uid1, dtype: int64
########## uid2
9500_321.0_150.0_226.0     14112
15885_545.0_185.0_138.0    10332
17188_321.0_150.0_226.0    10312
7919_194.0_150.0_166.0      8844
15066_170.0_150.0_102.0     7918
12695_490.0_150.0_226.0     7079
6019_583.0_150.0_226.0      6766
12544_321.0_150.0_226.0     6760
2803_100.0_150.0_226.0      6126
7919_194.0_150.0_202.0      6047
Name: uid2, dtype: int64
########## uid3
15885_545.0_185.0_138.0_nan_nan       9900
17188_321.0_150.0_226.0_299.0_87.0    5862
12695_490.0_150.0_226.0_325.0_87.0    5766
9500_321.0_150.0_226.0_204.0_87.0     4647
3154_408.0_185.0_224.0_nan_nan        4398
12839_321.0_150.0_226.0_264.0_87.0    3538
16132_111.0_150.0_226.0_299.0_87.0    3523
15497_490.0_150.0_226.0_299.0_87

In [18]:
#4. period counts
for col in ['DT_M','DT_W','DT_D']:
    temp_df = pd.concat([train_df[[col]], test_df[[col]]])
    fq_encode = temp_df[col].value_counts().to_dict()
            
    train_df[col+'_total'] = train_df[col].map(fq_encode)
    test_df[col+'_total']  = test_df[col].map(fq_encode)
        
#User period counts
periods = ['DT_M','DT_W','DT_D']
i_cols = ['uid4','uid5','uid6','uid7','cid_1']
for period in periods:
    for col in i_cols:
        new_column = col + '_' + period
            
        temp_df = pd.concat([train_df[[col,period]], test_df[[col,period]]])
        temp_df[new_column] = temp_df[col].astype(str) + '_' + (temp_df[period]).astype(str)
        fq_encode = temp_df[new_column].value_counts().to_dict()
            
        train_df[new_column] = (train_df[col].astype(str) + '_' + train_df[period].astype(str)).map(fq_encode)
        test_df[new_column]  = (test_df[col].astype(str) + '_' + test_df[period].astype(str)).map(fq_encode)
        
        train_df[new_column] /= train_df[period+'_total']
        test_df[new_column]  /= test_df[period+'_total']
        


In [19]:
#5. Prepare bank type feature
for df in [train_df, test_df]:
    df['bank_type'] = df['card3'].astype(str) +'_'+ df['card5'].astype(str)

encoding_mean = {
    1: ['DT_D','DT_hour','_hour_dist','DT_hour_mean'],
    2: ['DT_W','DT_day_week','_week_day_dist','DT_day_week_mean'],
    3: ['DT_M','DT_day_month','_month_day_dist','DT_day_month_mean'],
    }

encoding_best = {
    1: ['DT_D','DT_hour','_hour_dist_best','DT_hour_best'],
    2: ['DT_W','DT_day_week','_week_day_dist_best','DT_day_week_best'],
    3: ['DT_M','DT_day_month','_month_day_dist_best','DT_day_month_best'],   
    }

train_df['DT_day_month'] = (train_df['DT'].dt.day).astype(np.int8)
test_df['DT_day_month'] = (test_df['DT'].dt.day).astype(np.int8)
# Some ugly code here (even worse than in other parts)
for col in ['card3','card5','bank_type']:
    for df in [train_df, test_df]:
        for encode in encoding_mean:
            encode = encoding_mean[encode].copy()
            new_col = col + '_' + encode[0] + encode[2]
            df[new_col] = df[col].astype(str) +'_'+ df[encode[0]].astype(str)

            temp_dict = df.groupby([new_col])[encode[1]].agg(['mean']).reset_index().rename(
                                                                    columns={'mean': encode[3]})
            temp_dict.index = temp_dict[new_col].values
            temp_dict = temp_dict[encode[3]].to_dict()
            df[new_col] = df[encode[1]] - df[new_col].map(temp_dict)

        for encode in encoding_best:
            encode = encoding_best[encode].copy()
            new_col = col + '_' + encode[0] + encode[2]
            df[new_col] = df[col].astype(str) +'_'+ df[encode[0]].astype(str)
            temp_dict = df.groupby([col,encode[0],encode[1]])[encode[1]].agg(['count']).reset_index().rename(
                                                                    columns={'count': encode[3]})

            temp_dict.sort_values(by=[col,encode[0],encode[3]], inplace=True)
            temp_dict = temp_dict.drop_duplicates(subset=[col,encode[0]], keep='last')
            temp_dict[new_col] = temp_dict[col].astype(str) +'_'+ temp_dict[encode[0]].astype(str)
            temp_dict.index = temp_dict[new_col].values
            temp_dict = temp_dict[encode[1]].to_dict()
            df[new_col] = df[encode[1]] - df[new_col].map(temp_dict)

In [20]:
#6. BankType timeblock_frequency_encoding
i_cols = ['bank_type'] 
periods = ['DT_M','DT_W','DT_D']

# We have few options to encode it here:
# - Just count transactions
# (but some timblocks have more transactions than others)
# - Devide to total transactions per timeblock (proportions)
# - Use both
# - Use only proportions
train_df, test_df = timeblock_frequency_encoding(train_df, test_df, periods, i_cols, 
                                 with_proportions=False, only_proportions=True)

In [ ]:
def timehist1_2(col,product):
    N = 8000 if col in ['TransactionAmt'] else 9999999999999999 # clip trans amount for better view
    train_df[(train_df['isFraud'] == 0) & (train_df['ProductCD'] == product)].set_index('TransactionDT')[col].clip(0, N).plot(style='.', title='Hist ' + col, figsize=(15, 3))
    train_df[(train_df['isFraud'] == 1) & (train_df['ProductCD'] == product)].set_index('TransactionDT')[col].clip(0, N).plot(style='.', title='Hist ' + col, figsize=(15, 3))
    test_df[test_df['ProductCD'] == product].set_index('TransactionDT')[col].clip(0, N).plot(style='.', title=col + ' values over time (blue=no-fraud, orange=fraud, green=test)', figsize=(15, 3))
    plt.show()
%matplotlib inline
import matplotlib.pyplot as plt
import seaborn as sns
products=train_df.ProductCD.unique().tolist()
col='D1'
for prod in products: 
    print("Product code:", prod)
    timehist1_2(col, prod)

In [ ]:
#just make use of Ds with ProductCD
train_df['TransactionDTday'] = (train_df['TransactionDT']/(60*60*24)).map(int)
test_df['TransactionDTday'] = (test_df['TransactionDT']/(60*60*24)).map(int)
train_df['D1minusday'] = train_df['D1'] - train_df['TransactionDTday']
test_df['D1minusday'] = test_df['D1'] - test_df['TransactionDTday']
#train_df[(train_df.isFraud==1) & (train_df.D1minusday==78)][['card1','card2','card3','card4','card5','card6','addr1','addr2','dist1','dist2','P_emaildomain','R_emaildomain','TransactionDTday']]
col='D1minusday'
for prod in ['W','H','C','S','R']: 
    print("Product code:", prod)
    timehist1_2(col, prod)
train_df = train_df.drop(['D1minusday','TransactionDTday'],axis=1)
test_df = test_df.drop(['D1minusday','TransactionDTday'],axis=1)

In [ ]:
WHCSR = {'W':1,'H':0,'C':1,'S':1,'R':0}
def uid_CD_aggregation(train_df, test_df, main_columns, uids, aggregations):
    train_df['TransactionDTday'] = (train_df['TransactionDT']/(60*60*24)).map(int)
    test_df['TransactionDTday'] = (test_df['TransactionDT']/(60*60*24)).map(int)
    train_df['productCD_sep'] = train_df['ProductCD'].map(WHCSR)
    test_df['productCD_sep'] = test_df['ProductCD'].map(WHCSR)
    for main_column in main_columns:  
        train_df['Dminusday'] = train_df[main_column] - train_df['TransactionDTday']
        test_df['Dminusday'] = test_df[main_column] - test_df['TransactionDTday']
        for col in uids:
            for agg_type in aggregations:
                new_col_name = col+'_'+main_column+'_minusday_'+agg_type
                Dminus = [col,'Dminusday']
                temp_df = pd.concat([train_df[Dminus], test_df[Dminus]])
                temp_df = temp_df.groupby([col])['Dminusday'].agg([agg_type]).reset_index().rename(
                                                        columns={agg_type: new_col_name})

                temp_df.index = list(temp_df[col])
                temp_df = temp_df[new_col_name].to_dict()   

                train_df[new_col_name] = train_df[col].map(temp_df)
                test_df[new_col_name]  = test_df[col].map(temp_df)
                
    train_df = train_df.drop(['Dminusday','TransactionDTday'],axis=1)
    test_df = test_df.drop(['Dminusday','TransactionDTday'],axis=1)
    
    return train_df, test_df


In [21]:
#7. Ds uid aggregations (maybe not useful)
i_cols = ['D'+str(i) for i in range(1,16)]
uids = ['uid3','uid4','uid5','bank_type','cid1','uid6','uid7']
aggregations = ['mean','min']
'''
####### uIDs ProductCD aggregations
up_cols = ['D1','D2','D4','D5','D8','D10','D11','D15']
train_df, test_df = uid_CD_aggregation(train_df, test_df, up_cols, uids, aggregations)

#i_cols = [c for c in i_cols if c not in up_cols]
####### uIDs aggregations
train_df, test_df = uid_aggregation(train_df, test_df, i_cols, uids, aggregations)
'''
####### Cleaning Neagtive values and columns transformations
for df in [train_df, test_df]:

    for col in i_cols:
        df[col] = df[col].clip(0) 
    
    # Lets transform D8 and D9 column
    # As we almost sure it has connection with hours
    df['D9_not_na'] = np.where(df['D9'].isna(),0,1)
    df['D8_not_same_day'] = np.where(df['D8']>=1,1,0)
    df['D8_D9_decimal_dist'] = df['D8'].fillna(0)-df['D8'].fillna(0).astype(int)
    df['D8_D9_decimal_dist'] = ((df['D8_D9_decimal_dist']-df['D9'])**2)**0.5
    df['D8'] = df['D8'].fillna(-1).astype(int)

In [ ]:
#i_cols = ['D'+str(i) for i in range(1,16)]

In [22]:
def values_normalization(dt_df, periods, columns):
    for period in periods:
        for col in columns:
            new_col = col +'_'+ period
            dt_df[col] = dt_df[col].astype(float)  

            temp_min = dt_df.groupby([period])[col].agg(['min']).reset_index()
            temp_min.index = temp_min[period].values
            temp_min = temp_min['min'].to_dict()

            temp_max = dt_df.groupby([period])[col].agg(['max']).reset_index()
            temp_max.index = temp_max[period].values
            temp_max = temp_max['max'].to_dict()

            temp_mean = dt_df.groupby([period])[col].agg(['mean']).reset_index()
            temp_mean.index = temp_mean[period].values
            temp_mean = temp_mean['mean'].to_dict()

            temp_std = dt_df.groupby([period])[col].agg(['std']).reset_index()
            temp_std.index = temp_std[period].values
            temp_std = temp_std['std'].to_dict()

            dt_df['temp_min'] = dt_df[period].map(temp_min)
            dt_df['temp_max'] = dt_df[period].map(temp_max)
            dt_df['temp_mean'] = dt_df[period].map(temp_mean)
            dt_df['temp_std'] = dt_df[period].map(temp_std)

            dt_df[new_col+'_min_max'] = (dt_df[col]-dt_df['temp_min'])/(dt_df['temp_max']-dt_df['temp_min'])
            dt_df[new_col+'_std_score'] = (dt_df[col]-dt_df['temp_mean'])/(dt_df['temp_std'])
            del dt_df['temp_min'],dt_df['temp_max'],dt_df['temp_mean'],dt_df['temp_std']
    return dt_df

In [23]:
#8. Ds period calculation (maybe not useful)
#这个没有必要呢，LB也没有提高，CV有提升但看起来像过拟合……？

####### Values Normalization
i_cols.remove('D1')
i_cols.remove('D2')
i_cols.remove('D9')
periods = ['DT_D','DT_W','DT_M']

for df in [train_df, test_df]:
    df = values_normalization(df, periods, i_cols)


for col in ['D1','D2']:
    for df in [train_df, test_df]:
        df[col+'_scaled'] = df[col]/train_df[col].max()
        
####### Global Self frequency encoding
# self_encoding=True because 
# we don't need original values anymore
i_cols = ['D'+str(i) for i in range(1,16)]
train_df, test_df = frequency_encoding(train_df, test_df, i_cols, self_encoding=True)

In [24]:
#9. TransAmt uids/cids aggregations and calculations（need more fe）
i_cols = ['TransactionAmt','TransactionAmt_decimal']
#uids = ['card1','card2','card3','card5','uid1','uid2','uid3','uid4','uid5','bank_type','uid6']
uids = ['card1','card2','card3','card5','uid3','uid4','uid5','bank_type','uid6','uid7','cid_1']
aggregations = ['mean','std','min']

# uIDs aggregations
train_df, test_df = uid_aggregation(train_df, test_df, i_cols, uids, aggregations)

for df in [train_df,test_df]:
    df['transAmt_mut_C1'] = df['TransactionAmt'] * df['C1']
    df['transAmt_mut_C13'] = df['TransactionAmt'] * df['C13']
    df['transAmt_mut_C14'] = df['TransactionAmt'] * df['C14']
    df['transAmt_dec_diff'] = df['TransactionAmt_decimal'] - ((df['uid4_TransactionAmt_mean']-df['uid4_TransactionAmt_mean'].astype(int)) * 1000).astype(int)
    df['Transdiff_in_uid'] = df['transAmt_dec_diff']*df['uid4_TransactionAmt_mean']/1000

# TransactionAmt Normalization-period scaling
periods = ['DT_D','DT_W','DT_M']
for df in [train_df, test_df]:
    df = values_normalization(df, periods, i_cols)
# Product type
train_df['product_type'] = train_df['ProductCD'].astype(str)+'_'+train_df['TransactionAmt'].astype(str)
test_df['product_type'] = test_df['ProductCD'].astype(str)+'_'+test_df['TransactionAmt'].astype(str)

i_cols = ['product_type']
periods = ['DT_D','DT_W','DT_M']
train_df, test_df = timeblock_frequency_encoding(train_df, test_df, periods, i_cols, 
                                                 with_proportions=False, only_proportions=True)
train_df, test_df = frequency_encoding(train_df, test_df, i_cols, self_encoding=True)

In [25]:
def column_value_freq(sel_col,cum_per):
    dfpercount = pd.DataFrame(columns=['col_name','num_values_'+str(round(cum_per,2))])
    for col in sel_col:
        col_value = train_df[col].value_counts(normalize=True)
        colpercount = pd.DataFrame({'value' : col_value.index,'per_count' : col_value.values})
        colpercount['cum_per_count'] = colpercount['per_count'].cumsum()
        if len(colpercount.loc[colpercount['cum_per_count'] < cum_per,] ) < 2:
            num_col_99 = len(colpercount.loc[colpercount['per_count'] > (1- cum_per),]) #返回大头
        else:
            num_col_99 = len(colpercount.loc[colpercount['cum_per_count']< cum_per,] ) #返回小头
        dfpercount=dfpercount.append({'col_name': col,'num_values_'+str(round(cum_per,2)): num_col_99},ignore_index = True)
    dfpercount['unique_values'] = train_df[sel_col].nunique().values
    dfpercount['unique_value_to_num_values'+str(round(cum_per,2))+'_ratio'] = 100 * (dfpercount['num_values_'+str(round(cum_per,2))]/dfpercount.unique_values)
    #dfpercount['percent_missing'] = percent_na(train_transaction[sel_col])['percent_missing'].round(3).values
    return dfpercount

In [26]:
#10. V cols
#Understand V cols
v_cols = ['V'+str(i) for i in range(1,340)]
cum_per = 0.965
colfreq=column_value_freq(v_cols,cum_per)
print(colfreq.head())
colfreq_bool = colfreq[colfreq.unique_values==2]['col_name'].values
colfreq_pseudobool = colfreq[(colfreq.unique_values !=2) & (colfreq['num_values_'+str(round(cum_per,2))] <= 2)]
colfreq_pseudobool_cat = colfreq_pseudobool[colfreq_pseudobool.unique_values <=15]['col_name'].values
colfreq_pseudobool_num = colfreq_pseudobool[colfreq_pseudobool.unique_values >15]['col_name'].values
colfreq_cat = colfreq[(colfreq.unique_values >15) & (colfreq['num_values_'+str(round(cum_per,2))] <= 15) & (colfreq['num_values_'+str(round(cum_per,2))]> 2)]['col_name'].values
colfreq_num = colfreq[colfreq['num_values_'+str(round(cum_per,2))]>15]['col_name'].values

  col_name num_values_0.96  unique_values unique_value_to_num_values0.96_ratio
0       V1               1              2                                   50
1       V2               2              9                              22.2222
2       V3               2             10                                   20
3       V4               2              7                              28.5714
4       V5               2              7                              28.5714


In [27]:
v_low2 = ['V304','V84','V252','V16','V194','V193','V17','V98','V15','V111','V101','V22','V18','V116','V334','V284',
 'V32','V191','V297','V328','V104','V110','V196','V269','V31','V50','V114','V21','V302','V325','V113','V121','V118',
 'V65','V14','V41','V240','V1','V89','V119','V27','V122','V68','V120','V241','V117','V305','V88','V107','V28']
low_bool = [c for c in v_low2 if (c in colfreq_bool)]
low_pesudoboolcat = [c for c in v_low2 if (c in colfreq_pseudobool_cat)]
low_pesudoboolnum = [c for c in v_low2 if (c in colfreq_pseudobool_num)]
low_cat = [c for c in v_low2 if (c in colfreq_cat)]
low_num = [c for c in v_low2 if (c in colfreq_num)]

In [28]:
colfreq_cat

array(['V96', 'V97', 'V99', 'V129', 'V135', 'V139', 'V140', 'V151',
       'V152', 'V167', 'V168', 'V170', 'V171', 'V176', 'V177', 'V178',
       'V179', 'V217', 'V218', 'V219', 'V221', 'V222', 'V228', 'V229',
       'V230', 'V231', 'V232', 'V233', 'V234', 'V280', 'V282', 'V283',
       'V285', 'V294', 'V319', 'V322', 'V323', 'V324', 'V326', 'V329',
       'V336', 'V337', 'V338', 'V339'], dtype=object)

In [29]:
colfreq_cat

array(['V96', 'V97', 'V99', 'V129', 'V135', 'V139', 'V140', 'V151',
       'V152', 'V167', 'V168', 'V170', 'V171', 'V176', 'V177', 'V178',
       'V179', 'V217', 'V218', 'V219', 'V221', 'V222', 'V228', 'V229',
       'V230', 'V231', 'V232', 'V233', 'V234', 'V280', 'V282', 'V283',
       'V285', 'V294', 'V319', 'V322', 'V323', 'V324', 'V326', 'V329',
       'V336', 'V337', 'V338', 'V339'], dtype=object)

In [ ]:
'''
#以Group的PCA作为新特征
#PCA/ICA for dimensionality reduction
from sklearn.decomposition import PCA, FastICA,SparsePCA,KernelPCA
from sklearn.decomposition import TruncatedSVD
from sklearn.manifold import TSNE
from sklearn.random_projection import GaussianRandomProjection
from sklearn.random_projection import SparseRandomProjection


v_group_numcat_1 = ['V17', 'V18']
v_group_numcat_2 = ['V37', 'V38', 'V39', 'V40', 'V44', 'V45']
v_group_numcat_3 = ['V55','V56', 'V59', 'V60']
v_group_numcat_4 = ['V77', 'V78', 'V80', 'V81', 'V86', 'V87']
v_group_numcat_5 = ['V95', 'V100', 'V101', 'V102', 'V103', 'V104', 'V105', 'V106','V96', 'V97', 'V99', 'V129', 'V135']
v_group_numcat_6 = ['V139', 'V140', 'V151','V152','V138', 'V146', 'V147', 'V148', 'V149', 'V153', 'V154', 'V155','V156', 'V157', 'V158', 'V161', 'V162', 'V163']
v_group_numcat_7 = ['V167', 'V168', 'V170', 'V171', 'V176', 'V177', 'V178','V179', 'V169', 'V172','V180', 'V181', 'V182', 'V183', 'V184', 'V185', 'V186', 'V187','V188', 'V189', 'V190', 'V191', 'V192', 'V193', 'V195', 'V196','V198', 'V199', 'V200', 'V201']
v_group_numcat_8 = ['V217', 'V218', 'V219', 'V221', 'V222', 'V228', 'V229','V230', 'V231', 'V232', 'V233', 'V234', 'V220', 'V223', 'V224', 'V225', 'V226', 'V227', 'V235', 'V236', 'V237', 'V238', 'V239', 'V242','V243', 'V244', 'V245', 'V246', 'V247', 'V248', 'V249', 'V250','V251', 'V252', 'V253', 'V254', 'V255', 'V256', 'V257', 'V258','V259', 'V261', 'V262', 'V269'] 
v_group_numcat_9 = ['V279', 'V281', 'V287', 'V290','V280', 'V282', 'V283','V285', 'V294', 'V319']
v_group_numcat_10 = ['V291', 'V292', 'V293', 'V295', 'V296', 'V298', 'V299', 'V302','V303', 'V304', 'V311']
v_group_numcat_11 = ['V327', 'V328', 'V330', 'V334','V322', 'V323', 'V324', 'V326', 'V329','V336', 'V337', 'V338', 'V339']
vs = [v_group_numcat_1,v_group_numcat_2,v_group_numcat_3,v_group_numcat_4,v_group_numcat_5,v_group_numcat_6,v_group_numcat_7,v_group_numcat_8,v_group_numcat_9,v_group_numcat_10,v_group_numcat_11]
for vcols in vs:
    print('now in the',vcols)
    train_df[vcols] = train_df[vcols].fillna(-999)
    test_df[vcols] = test_df[vcols].fillna(-999)
    #train_df[~train_df.isin([np.inf,np.nan])] = -999
    #test_df[~test_df.isin([np.inf,np.nan])] = -999
    
    #PCA
    print('PCA……')
    pca = PCA(n_components = 1,random_state=42)
    train_df['pca_' + vcols[0]] = pca.fit_transform(train_df[vcols])
    test_df['pca_' + vcols[0]] = pca.transform(test_df[vcols])

    # ICA
    print('ICA……')
    ica = FastICA(n_components = 1,random_state=42)
    train_df['ica_' + vcols[0]] = ica.fit_transform(train_df[vcols])
    test_df['ica_' + vcols[0]] = ica.transform(test_df[vcols])

    # GRP
    grp = GaussianRandomProjection(n_components = 1,eps=0.1, random_state=42)
    train_df['grp_' + vcols[0]] = grp.fit_transform(train_df[vcols])
    test_df['grp_' + vcols[0]] = grp.transform(test_df[vcols])

    # SRP
    srp = SparseRandomProjection(n_components = 1,dense_output=True, random_state=42)
    train_df['srp_' + vcols[0]] = srp.fit_transform(train_df[vcols])
    test_df['srp_' + vcols[0]] = srp.transform(test_df[vcols])
    
    # tSVD
    print('tSVD……')
    tsvd = TruncatedSVD(n_components = 1,random_state=42)
    train_df['tsvd_' + vcols[0]] = tsvd.fit_transform(train_df[vcols])
    test_df['tsvd_' + vcols[0]] = tsvd.transform(test_df[vcols])

    '''

In [30]:
#cliping v_num_cats
vcol_spike = ['V96', 'V97','V167', 'V168','V177', 'V178','V179', 'V217', 'V218', 'V219','V231','V280', 'V282','V294', 'V322', 'V323', 'V324']
cols = list(colfreq_pseudobool_num) + vcol_spike
for df in [train_df, test_df]:
    for col in cols :
        max_value = train_df[train_df['DT_M']==train_df['DT_M'].min()][col].max()
        df[col] = df[col].clip(None,max_value) 


In [31]:
#Dealing with V cols
#Scaling with pca - Numerical V cols - scaling仍需谨慎
from sklearn.preprocessing import StandardScaler

v_cols = colfreq_num
print(v_cols)
test_group = list(v_cols)
train_df['group_sum'] = train_df[test_group].to_numpy().sum(axis=1)
train_df['group_mean'] = train_df[test_group].to_numpy().mean(axis=1)
    
test_df['group_sum'] = test_df[test_group].to_numpy().sum(axis=1)
test_df['group_mean'] = test_df[test_group].to_numpy().mean(axis=1)
compact_cols = ['group_sum','group_mean']
 
for col in test_group:
    sc = StandardScaler()
    sc.fit(train_df[[col]].fillna(0))
    train_df[col] = sc.transform(train_df[[col]].fillna(0))
    test_df[col] = sc.transform(test_df[[col]].fillna(0))
    
sc_test_group = test_group

# check -> same obviously
features_check = []
from scipy.stats import ks_2samp #检查两个分布是否相同的函数
for col in sc_test_group:
    features_check.append(ks_2samp(train_df[col], test_df[col])[1])
    
features_check = pd.Series(features_check, index=sc_test_group).sort_values() 
print(features_check)

from sklearn.decomposition import PCA
#PCA还是必要的-是正交线性去噪
pca = PCA(random_state=42)
pca.fit(train_df[sc_test_group])
print(len(sc_test_group), pca.transform(train_df[sc_test_group]).shape[-1])
train_df[sc_test_group] = pca.transform(train_df[sc_test_group])
test_df[sc_test_group] = pca.transform(test_df[sc_test_group])

sc_variance =pca.explained_variance_ratio_
print(sc_variance)

# check
features_check = []

for col in sc_test_group:
    features_check.append(ks_2samp(train_df[col], test_df[col])[1])
    
features_check = pd.Series(features_check, index=sc_test_group).sort_values() 
print(features_check)
train_df[col], test_df[col]

['V126' 'V127' 'V128' 'V130' 'V131' 'V132' 'V133' 'V134' 'V136' 'V137'
 'V143' 'V144' 'V145' 'V150' 'V159' 'V160' 'V164' 'V165' 'V166' 'V202'
 'V203' 'V204' 'V205' 'V206' 'V207' 'V208' 'V209' 'V210' 'V211' 'V212'
 'V213' 'V214' 'V215' 'V216' 'V263' 'V264' 'V265' 'V266' 'V267' 'V268'
 'V270' 'V271' 'V272' 'V273' 'V274' 'V275' 'V276' 'V277' 'V278' 'V306'
 'V307' 'V308' 'V309' 'V310' 'V312' 'V313' 'V314' 'V315' 'V316' 'V317'
 'V318' 'V320' 'V321' 'V331' 'V332' 'V333' 'V335']
V130    1.069942e-100
V136     3.228515e-87
V317     1.542568e-65
V133     9.980904e-61
V127     1.833679e-60
            ...      
V206     2.495735e-01
V332     2.967873e-01
V333     2.998484e-01
V331     4.952229e-01
V335     5.364810e-01
Length: 67, dtype: float64
67 67
[3.95243705e-01 1.20604713e-01 9.08724136e-02 7.99145695e-02
 5.93916129e-02 5.00332241e-02 4.54584312e-02 2.89428818e-02
 2.32736617e-02 1.84120687e-02 1.45453003e-02 1.14526355e-02
 7.81445065e-03 7.34786437e-03 5.85068362e-03 4.37949141e-03
 4.0

(0         0.000023
 1         0.000009
 2         0.000009
 3         0.000015
 4         0.000141
             ...   
 590535    0.000013
 590536    0.000009
 590537    0.000009
 590538    0.000012
 590539    0.000016
 Name: V335, Length: 590540, dtype: float64, 0         0.000009
 1        -0.000035
 2        -0.000038
 3         0.000018
 4        -0.000004
             ...   
 506686    0.000009
 506687    0.000007
 506688    0.000009
 506689    0.000009
 506690    0.000009
 Name: V335, Length: 506691, dtype: float64)

In [ ]:
#11. Wanna some lag features
'''
import multiprocessing
from multiprocessing import Pool, cpu_count

def create_lag(df_code):
    rollings=['60d','90d']
    for rolling_type in rollings:
        df_code.index = df_code.DT
        rolled = df_code.drop(['DT'],axis=1).rolling(rolling_type)
        df_code = df_code.join(rolled.median().add_suffix(f'_lag_{rolling_type}_median'))
        df_code = df_code.join(rolled.max().add_suffix(f'_lag_{rolling_type}_max'))
        df_code = df_code.join(rolled.mean().add_suffix(f'_lag_{rolling_type}_mean'))
    return df_code.reset_index(drop=True)


def uid_rolling(df,main_columns,uidi):
    N_THREADS=multiprocessing.cpu_count()
    df_codes = df.groupby([uidi])
    all_df = []
    df_codes = [df_code[1][['DT']+main_columns] for df_code in df_codes]
    pool = Pool(N_THREADS)
    all_df = pool.map(create_lag, df_codes)
    new_df = pd.concat(all_df)  
    new_df.drop(main_columns,axis=1,inplace=True)
    pool.close()
    
    return new_df
'''

In [ ]:
'''
i_cols=['C1','C2','C4','C6','C7','C8','C10','C11','C12','C13','C14','V144','V145','V150','V159','V160']
uids = ['uid1','uid2','uid3','uid4','uid5','bank_type']
for col in uids:
    train_df = uid_rolling(train_df, i_cols, col)
    test_df = uid_rolling(test_df, i_cols, col)
'''

In [ ]:
#train_df = train_df.reset_index(drop=True)
#test_df = test_df.reset_index(drop=True)

In [32]:
#12. Cs frequency encode and clip
i_cols = ['C'+str(i) for i in range(1,15)]

####### Global Self frequency encoding
# self_encoding=False because 
# I want to keep original values
train_df, test_df = frequency_encoding(train_df, test_df, i_cols, self_encoding=False)

####### uIDs aggregations
#i_cols = ['C1','C2','C4','C6','C7','C11','C12','C14']
#train_df, test_df = uid_aggregation(train_df, test_df, i_cols, uids, aggregations)

####### Clip max values-这就跟丢掉冬天一样了
for df in [train_df, test_df]:
    for col in i_cols:
        max_value = train_df[train_df['DT_M']==train_df['DT_M'].max()][col].max()
        df[col] = df[col].clip(None,max_value) 

In [ ]:
#想查看下分布呢
%matplotlib inline
import matplotlib.pyplot as plt
import seaborn as sns
sns.distplot(train_df['V145'], kde=True, rug=True)

In [ ]:
sns.distplot(test_df['V145'],kde=True, rug=True)

In [ ]:
#对看起来没处理好的Vcols做一下处理：
'''
train_df[[f'V{i}_diff1' for i in range(126, 138)]] = train_df.groupby(['card1', 'ProductCD', 'addr1'])[[f'V{i}' for i in range(126, 138)]].diff() 
test_df[[f'V{i}_diff1' for i in range(126, 138)]] = test_df.groupby(['card1', 'ProductCD', 'addr1'])[[f'V{i}' for i in range(126, 138)]].diff() 
'''

In [33]:
#13. More combinations
## Identity columns
from sklearn.preprocessing import LabelEncoder
for col in ['id_33']:
    train_identity[col] = train_identity[col].fillna('unseen_before_label')
    test_identity[col]  = test_identity[col].fillna('unseen_before_label')
    
    le = LabelEncoder()
    le.fit(list(train_identity[col])+list(test_identity[col]))
    train_identity[col] = le.transform(train_identity[col])
    test_identity[col]  = le.transform(test_identity[col])
    
print('train_set shape before merge:',train_df.shape)
train_df1 = train_df.merge(train_identity,how='left',on=['TransactionID'])
print('train_set shape after merge:',train_df.shape)

print('test_set shape before merge:',test_df.shape)
test_df1 = test_df.merge(test_identity,how='left',on=['TransactionID'])
print('test_set shape after merge:',test_df.shape)

# New feature - mean of sth
columns_a = ['TransactionAmt', 'id_02', 'D15']
columns_b = ['card1', 'card4', 'addr1']
for col_a in columns_a:
    for col_b in columns_b:
        for df in [train_df1, test_df1]:
            df[f'{col_a}_to_mean_{col_b}'] = df[col_a] / df.groupby([col_b])[col_a].transform('mean')
            df[f'{col_a}_to_std_{col_b}'] = df[col_a] / df.groupby([col_b])[col_a].transform('std')
del columns_a,columns_b
gc.collect()

# Some arbitrary features interaction 试做联合特征(?????)
for feature in ['id_02__id_20', 'id_02__D8', 'D11__DeviceInfo', 'DeviceInfo__P_emaildomain', 'P_emaildomain__C2', 
                    'card2__dist1', 'card1__card5', 'card2__id_20', 'card5__P_emaildomain', 'addr1__card1','card1__id_02']:

    f1, f2 = feature.split('__')
    train_df1[feature] = train_df1[f1].astype(str) + '_' + train_df1[f2].astype(str)
    test_df1[feature] = test_df1[f1].astype(str) + '_' + test_df1[f2].astype(str)

    le = LabelEncoder()
    le.fit(list(train_df1[feature].astype(str).values) + list(test_df1[feature].astype(str).values))
    train_df1[feature] = le.transform(list(train_df1[feature].astype(str).values))
    test_df1[feature] = le.transform(list(test_df1[feature].astype(str).values))

train_set shape before merge: (590540, 665)
train_set shape after merge: (590540, 665)
test_set shape before merge: (506691, 665)
test_set shape after merge: (506691, 665)


In [34]:
train_df = train_df1
test_df = test_df1

In [35]:
train_df['had_id'] = train_df['had_id'].fillna(0)
test_df['had_id'] = test_df['had_id'].fillna(0)
def uid_sep_aggregation(train_df, test_df, main_columns, uids, aggregations):
    for main_column in main_columns:  
        for col in uids:
            for agg_type in aggregations:
                new_col_name = col+'_'+main_column+'_sep_'+agg_type
                
                train_df[col+'_sep'] = train_df[col].astype(str)+train_df['had_id'].astype(str)
                test_df[col+'_sep'] = test_df[col].astype(str)+test_df['had_id'].astype(str)
                
                temp_df = pd.concat([train_df[[col+'_sep', main_column]], test_df[[col+'_sep',main_column]]])
                temp_df = temp_df.groupby([col+'_sep'])[main_column].agg([agg_type]).reset_index().rename(
                                                        columns={agg_type: new_col_name})
                
                temp_df.index = list(temp_df[col+'_sep'])
                temp_df = temp_df[new_col_name].to_dict()   
                
                train_df[new_col_name] = train_df[col+'_sep'].map(temp_df)
                test_df[new_col_name]  = test_df[col+'_sep'].map(temp_df)
                del train_df[col+'_sep'],test_df[col+'_sep']
    return train_df, test_df

def values_sep_normalization(dt_df, periods, columns):
    for period in periods:
        for col in columns:
            new_col = col +'_sep_'+ period
            dt_df[col] = dt_df[col].astype(float)  

            dt_df[period+'_sep'] = dt_df[period].astype(str)+dt_df['had_id'].astype(str)     
                
            temp_min = dt_df.groupby([period+'_sep'])[col].agg(['min']).reset_index()
            temp_min.index = temp_min[period+'_sep'].values
            temp_min = temp_min['min'].to_dict()

            temp_max = dt_df.groupby([period+'_sep'])[col].agg(['max']).reset_index()
            temp_max.index = temp_max[period+'_sep'].values
            temp_max = temp_max['max'].to_dict()

            temp_mean = dt_df.groupby([period+'_sep'])[col].agg(['mean']).reset_index()
            temp_mean.index = temp_mean[period+'_sep'].values
            temp_mean = temp_mean['mean'].to_dict()

            temp_std = dt_df.groupby([period+'_sep'])[col].agg(['std']).reset_index()
            temp_std.index = temp_std[period+'_sep'].values
            temp_std = temp_std['std'].to_dict()

            dt_df['temp_min'] = dt_df[period+'_sep'].map(temp_min)
            dt_df['temp_max'] = dt_df[period+'_sep'].map(temp_max)
            dt_df['temp_mean'] = dt_df[period+'_sep'].map(temp_mean)
            dt_df['temp_std'] = dt_df[period+'_sep'].map(temp_std)

            dt_df[new_col+'_min_max'] = (dt_df[col]-dt_df['temp_min'])/(dt_df['temp_max']-dt_df['temp_min'])
            dt_df[new_col+'_std_score'] = (dt_df[col]-dt_df['temp_mean'])/(dt_df['temp_std'])
            del dt_df['temp_min'],dt_df['temp_max'],dt_df['temp_mean'],dt_df['temp_std'],dt_df[period+'_sep']
    return dt_df

In [36]:
#9.1 TransAmt seperated by had_id(Online/Traditional)
#分online/traditional来groupbyuid
i_cols = ['TransactionAmt','TransactionAmt_decimal']
uids = ['uid3','uid4','uid5','bank_type','uid6','uid7','cid_1']
aggregations = ['mean','std','min']

train_df, test_df = uid_aggregation(train_df, test_df, i_cols, uids, aggregations)

#分online/traditional来normalization
periods = ['DT_D','DT_W','DT_M']
for df in [train_df, test_df]:
    df = values_sep_normalization(df, periods, i_cols)


In [37]:
#14. Category Encoding
print('Category Encoding')
from sklearn.preprocessing import LabelEncoder
## card4, card6, ProductCD
# Converting Strings to ints(or floats if nan in column) using frequency encoding
# We will be able to use these columns as category or as numerical feature


for col in ['card4', 'card6', 'ProductCD']:
    print('Encoding', col)
    temp_df = pd.concat([train_df[[col]], test_df[[col]]])
    col_encoded = temp_df[col].value_counts().to_dict()   
    train_df[col] = train_df[col].map(col_encoded) #多分类用出现次数作为编码
    test_df[col]  = test_df[col].map(col_encoded)
    print(col_encoded)
    del temp_df,col_encoded
    gc.collect()

## M columns
# Converting Strings to ints(or floats if nan in column)

for col in ['M1','M2','M3','M5','M6','M7','M8','M9']:
    train_df[col] = train_df[col].map({'T':1, 'F':0})
    test_df[col]  = test_df[col].map({'T':1, 'F':0})

for col in ['P_emaildomain', 'R_emaildomain','M4']:
    print('Encoding', col)
    temp_df = pd.concat([train_df[[col]], test_df[[col]]])
    col_encoded = temp_df[col].value_counts().to_dict()   
    train_df[col] = train_df[col].map(col_encoded)
    test_df[col]  = test_df[col].map(col_encoded)
    print(col_encoded)
    del temp_df,col_encoded
    gc.collect()
    
i_cols = ['TransactionAmt']
uids = ['card2__id_20','card1__id_02']
aggregations = ['mean','std']

# uIDs aggregations
train_df, test_df = uid_aggregation(train_df, test_df, i_cols, uids, aggregations)
 
    
## Reduce Mem One More Time
train_df = reduce_mem_usage(train_df)
test_df  = reduce_mem_usage(test_df)

Category Encoding
Encoding card4
{'visa': 722693, 'mastercard': 348803, 'american express': 16078, 'discover': 9572}
Encoding card6
{'debit': 828379, 'credit': 268753, 'charge card': 16}
Encoding ProductCD
{'W': 800657, 'C': 137785, 'R': 73346, 'H': 62397, 'S': 23046}
Encoding P_emaildomain
{'gmail.com': 435803, 'yahoo.com': 182784, 'uknown': 163648, 'hotmail.com': 85649, 'anonymous.com': 71062, 'aol.com': 52337, 'comcast.net': 14474, 'icloud.com': 12316, 'outlook.com': 9934, 'att.net': 7647, 'msn.com': 7480, 'sbcglobal.net': 5767, 'live.com': 5720, 'verizon.net': 5011, 'ymail.com': 4075, 'bellsouth.net': 3437, 'yahoo.com.mx': 2827, 'me.com': 2713, 'cox.net': 2657, 'optonline.net': 1937, 'live.com.mx': 1470, 'charter.net': 1443, 'mail.com': 1156, 'rocketmail.com': 1105, 'gmail': 993, 'earthlink.net': 979, 'outlook.es': 863, 'mac.com': 862, 'hotmail.fr': 674, 'hotmail.es': 627, 'frontier.com': 594, 'roadrunner.com': 583, 'juno.com': 574, 'windstream.net': 552, 'web.de': 518, 'aim.com': 

In [ ]:
train_df = reduce_mem_usage(train_df)
test_df  = reduce_mem_usage(test_df)

In [38]:
## Export
train_df.to_pickle('train_transaction_15.pkl')
test_df.to_pickle('test_transaction_15.pkl')

#train_identity.to_pickle('train_identity_new01.pkl')
#test_identity.to_pickle('test_identity_new01.pkl')

In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt
import seaborn as sns
#c_features = ['V'+str(i) for i in range(103,105)]
#c_features = [c for c in c_features if c not in v_low2]
c_features = ['V17']
for i in c_features:
    cor = np.corrcoef(train_df['TransactionDT'], train_df[i])[0,1]
    train_df.set_index('TransactionDT')[i].plot(style='.', title=i+" corr= "+str(round(cor,3)), figsize=(15, 3))
    test_df.set_index('TransactionDT')[i].plot(style='.', title=i+" corr= "+str(round(cor,3)), figsize=(15, 3))
    plt.show()

In [ ]:
print(train_df.columns.values)

In [ ]:
train_df[train_df['is_december']==1]['DT_M'].head()

In [ ]:
test_df[test_df['is_december']==1]['DT_M'].tail()

In [ ]:
test_df[train_df['C14']==1429]['DT_day_month']

In [ ]:
train_df['duplicate_count']#看起来是个挺好的特征呀

In [ ]:
test_df['had_id'].isnull().sum()/test_df.shape[0]